# Day 2 lunch exerceise

## 1. Mapping FlyBase genes to UniProt

### Create a new mapping file with headers.

In [9]:
id_map = open('id_map.txt', 'w')
id_map.write('FlyBaseID\tUniProtID\n')
id_map.close()

### Extract FlyBase ID and UniProt ID from fly.txt

In [10]:
fly = open('fly.txt', 'r')

In [11]:
for line in fly:
    if 'DROME' not in line or 'FBgn' not in line:     #Some entries do not have a FBgn number, so exclude them.
        continue
    content = line.rstrip().split()
    content = content[3] + '\t' + content[2] + '\n'
    id_map = open('id_map.txt', 'a')
    id_map.write(content)
    id_map.close()

## 2. ID mapping

## Define the function

In [72]:
def IDmapping(ctabfile, mapfile, outputfname, ignore = True, notfound = '*'):
    '''Maps the FlyBase Number in ctabfile to the UniProt Number in mapfile. 
        This function gives the first 100 entries. 
        If a FlyBase Number is not present in the map file, it is skipped or replaced by a certain sign based on the optional variables.'''
    #Create the output file with a header.
    uniP_ctab = open(outputfname, 'w')
    uniP_ctab.write('t_id\tchr\tstrand\tstart\tend\tt_name\tnum_exons\tlength\tUniProt_id\tgene_name\tcov\tFPKM\n')
    uniP_ctab.close()
    
    #Create a dictionary for mapping
    mapping = dict()
    id_map = open(mapfile, 'r')
    for line in id_map:
        if line[1] == 'l':
            continue
        IDs = line.rstrip().split()
        mapping[IDs[0]] = IDs[1]
    id_map.close()
    
    #Map gene IDs.
    ctab = open(ctabfile, 'r')
    ctab.seek(0, 0)
    n = 0
    for line in ctab:
        if n == 0:
            n += 1
            continue
        elif n == 101:
            break
        content = line.rstrip().split()
        try:
            content[8] = mapping[content[8]]
        except:
            if ignore:
                #n += 1     #If we only need information about the first 100 entries in the ctab file, add this line.
                continue
            content[8] = notfound
        content = '\t'.join(content)
        content += '\n'
        uniP_ctab = open(outputfname, 'a')
        uniP_ctab.write(content)
        uniP_ctab.close()
        n += 1
    ctab.close()
    return

## Ignore the not-found entries

In [73]:
IDmapping('../../data/results/stringtie/SRR072893/t_data.ctab', 'id_map.txt', 'UniP_t_data_ignore.txt')

## Use '\*' to replace the not-found FlyBase Numbers

In [75]:
IDmapping('../../data/results/stringtie/SRR072893/t_data.ctab', 'id_map.txt', 'UniP_t_data_notignore.txt', False)